In [ ]:
import os, sys

import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import mlflow
from mlflow.models.signature import infer_signature
from dotenv import load_dotenv, find_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate, cross_val_score
from xgboost import XGBRegressor

cur_dir = os.getcwd()
SRC_PATH = cur_dir[: cur_dir.index("fortunato-wheels-engine") + len("fortunato-wheels-engine")]
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from src.data.car_ads import CarAds
from src.logs import get_logger
from src.data.training_preprocessing import preprocess_ads_for_training

logger = get_logger(__name__)

AZURE_MLFLOW_URI = os.environ.get("AZURE_MLFLOW_URI")
mlflow.set_tracking_uri(AZURE_MLFLOW_URI)

sns.set_theme(style="whitegrid")
sns.set(rc={"figure.figsize": (8, 12)})
# set context to notebook
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams["font.family"] = "sans serif"

%load_ext autoreload
%autoreload 2

## Load in current car adds

In [ ]:
ads = CarAds()
# ads.get_car_ads(sources=["cargurus", "kijiji"])
ads.get_car_ads(data_dump=os.path.join(SRC_PATH, "data", "processed", "car-ads-dump_2023-07-18.csv"))

In [ ]:
ads.df.info()

In [ ]:
ads.df.describe()

In [ ]:
ads.preprocess_ads()

In [ ]:
model_features = [
    "age_at_posting",
    "mileage_per_year",
    "make",
    "model",
    "price",
    "wheel_system"
]

preprocessed_ads = preprocess_ads_for_training(ads.df, model_features=model_features)

train_df, test_df = train_test_split(
    preprocessed_ads,
    test_size=0.2,
    random_state=42,
    stratify=preprocessed_ads["model"],
)

# with features selected drop all with null values
train_df = train_df[model_features].dropna().reset_index(drop=True)
test_df = test_df[model_features].dropna().reset_index(drop=True)

X_train = train_df.drop(columns=["price"])
y_train = train_df["price"]
X_test = test_df.drop(columns=["price"])
y_test = test_df["price"]

In [ ]:
# plot how many ads there are by the top make_name values
fig = px.histogram(
    # ads.loc[ads.make_name.isin(ads.make_name.value_counts().index[:15])],
    train_df.loc[train_df.model.isin(train_df.model.value_counts().index[:30])],
    x="model",
    title="Number of Ads by Model",
    color="model",
    labels={"model": "Model"},
    color_discrete_sequence=px.colors.qualitative.Dark24,
    height=500,
    category_orders={"model": train_df.model.value_counts().index[:30]}
)
fig.show()

In [ ]:
# a bar chart of what percentage of all ads is made up by the top 20 makes
fig = px.bar(
    train_df.make.value_counts(normalize=True)[:20]*100,
    color=train_df.make.value_counts(normalize=True)[:20].index,
    title="TRAIN SET: Percentage of Ads by Make, Top 20 Makes",
    labels={"index": "Make", "value": "Percentage of Ads"},
    height=500,
    color_discrete_sequence=px.colors.qualitative.Dark24,
    text=train_df.make.value_counts(normalize=True)[:20].apply(lambda x: f"{x*100:.1f}%")
)
fig.show()

In [ ]:
# a bar chart of what percentage of all ads is made up by the bottom 20 makes
fig = px.bar(
    train_df.make.value_counts(normalize=True)[-20:]*100,
    color=train_df.make.value_counts(normalize=True)[-20:].index,
    title="TRAIN SET: Percentage of Ads by Make, Bottom 20 Makes",
    labels={"index": "Make", "value": "Percentage of Ads"},
    height=500,
    color_discrete_sequence=px.colors.qualitative.Dark24,
    text=train_df.make.value_counts(normalize=True)[-20:].apply(lambda x: f"{x*100:.1f}%")
)
fig.show()

In [ ]:
fig = px.bar(
    train_df.make.value_counts()[:20],
    color=train_df.make.value_counts(normalize=True)[:20].index,
    title="TRAIN SET: Number of Ads by Make, Top 20 Makes",
    labels={"index": "Make", "value": "Number of Ads"},
    height=500,
    color_discrete_sequence=px.colors.qualitative.Dark24,
    text=train_df.make.value_counts()[:20].apply(lambda x: f"{x/1000:.0f}k")
)
fig.show()

In [ ]:
fig = px.bar(
    train_df.make.value_counts()[-20:],
    color=train_df.make.value_counts(normalize=True)[-20:].index,
    title="TRAIN SET: Number of Ads by Make, Bottom 20 Makes",
    labels={"index": "Make", "value": "Number of Ads"},
    height=500,
    color_discrete_sequence=px.colors.qualitative.Dark24,
    text=train_df.make.value_counts()[-20:].apply(lambda x: f"{x/1000:.1f}k")
)
fig.show()

## Load and Fit model

In [ ]:
import os
from azureml.core import Workspace
from azureml.core.model import Model

# what version of the model to download/evaluate
PRICE_PREDICTION_MODEL_VER = 3
PRICE_PREDICTION_MODEL_PATH = os.path.join(
    os.pardir, "models", "all-vehicles-price-prediction", str(PRICE_PREDICTION_MODEL_VER)
)

In [ ]:
ws = Workspace.from_config(
    # assumed running from root of repo
    path=os.path.join(os.pardir, "src", "deployment", "price_prediction_api", "config.json")
)

price_model = Model(
    ws, "all-vehicles-price-prediction", version=PRICE_PREDICTION_MODEL_VER
)

price_model.download(
    target_dir=PRICE_PREDICTION_MODEL_PATH,
    exist_ok=True,
)

In [ ]:
# Load the model from file
model_path = os.path.join(PRICE_PREDICTION_MODEL_PATH, "model", "model.pkl")

with open(model_path, 'rb') as f:
    model = pickle.load(f)


In [ ]:
# Predict on test set
y_pred = model.predict(X_test)

# Score model (MAPE, RMSE, r2) 
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"MAPE: {mape:.2f}%")
print(f"RMSE: ${rmse:.0f}")
print(f"r2: {r2:.3f}")

In [ ]:
# add predicted price to test_df, round to 1 decimal place
full_df = test_df.copy(deep=True)
full_df['predicted_price'] = y_pred.round(1)
full_df

In [ ]:
fig = px.scatter(
    full_df, 
    x='price', 
    y='predicted_price', 
    opacity=0.5,
)

fig.update_layout(
    xaxis_title="Actual Prices",
    yaxis_title="Predicted Prices",
    title="Scatter plot: Actual Prices vs. Predicted Prices",
    width=800,
    height=800,
)
# add a diagonal black line to show where predicted = actual
fig.add_shape(
    type="line",
    x0=0,
    y0=0,
    x1=full_df.price.max(),
    y1=full_df.price.max(),
    line=dict(
        color="Black",
        width=2,
        dash="dashdot",
    )
)
fig.show()

In [ ]:
fig = px.scatter(full_df.loc[full_df.make.isin(full_df.make.value_counts().index[-20:])], 
                x='price',
                y='predicted_price', 
                hover_data=['model', 'mileage_per_year', 'age_at_posting', 'wheel_system'],
                color='make',
                opacity=0.5,
                facet_col='make',
                facet_col_wrap=5
                )
fig.update_layout(
    xaxis_title="Actual Prices",
    yaxis_title="Predicted Prices",
    title="Scatter plot: Actual Prices vs. Predicted Prices",
    # width=800,
    height=1200,
)
fig.show()

In [ ]:
# get unique make model combinations
unique_make_models = full_df[['make', 'model']].drop_duplicates()
unique_makes = full_df['make'].unique()

mape_by_make_model = {}
rmse_by_make_model = {}

mape_by_make = {}
rmse_by_make= {}

# calcualte metrics for each make model combination
for make, model in unique_make_models.values:
    subset_data = full_df[(full_df['make'] == make) & (full_df['model'] == model)]
    y_actual_subset = subset_data['price']
    y_predicted_subset = subset_data['predicted_price']
        
    mape = np.mean(np.abs((y_actual_subset - y_predicted_subset) / y_actual_subset)) * 100 
    rmse = np.sqrt(mean_squared_error(y_actual_subset, y_predicted_subset))
        
    mape_by_make_model[(make, model)] = mape
    rmse_by_make_model[(make, model)] = rmse
    
# calcualte metrics for each make
for make in unique_makes:
    subset_data = full_df[full_df['make'] == make]
    y_actual_subset = subset_data['price']
    y_predicted_subset = subset_data['predicted_price']
        
    mape = np.mean(np.abs((y_actual_subset - y_predicted_subset) / y_actual_subset)) * 100 
    rmse = np.sqrt(mean_squared_error(y_actual_subset, y_predicted_subset))
     
    mape_by_make[make] = mape
    rmse_by_make[make] = rmse


# save results to dataframes
make_model_metrics = (
    pd.DataFrame({'MAPE': mape_by_make_model, 'RMSE': rmse_by_make_model})
    .reset_index()
    .rename(columns={'level_0': 'make', 'level_1': 'model'})
)

make_metrics = (
    pd.DataFrame({'MAPE': mape_by_make, 'RMSE': rmse_by_make})
    .reset_index()
    .rename(columns={'index': 'make'})
)

In [ ]:

# plot MAPE and RMSE by make
fig_mape = px.bar(make_metrics,
                x='make', 
                y='MAPE',
                color='make',
                )
fig_mape.update_layout(title='MAPE by Make', 
                       xaxis_title='Make', 
                       yaxis_title='MAPE', 
                       font=dict(size=14),
                       xaxis={'categoryorder':'total descending'})

fig_rmse = px.bar(make_metrics,
                x='make', 
                y='RMSE',
                color='make',
                )
fig_rmse.update_layout(title='RMSE by Make', 
                       xaxis_title='Make', 
                       yaxis_title='RMSE', 
                       font=dict(size=14),
                       xaxis={'categoryorder':'total descending'})

fig_mape.show()
fig_rmse.show()

In [ ]:
# plot diagonal plot of Scion and Pontiac predicted vs actual prices
makes_to_investigate = ['Scion', 'Pontiac']

fig = px.scatter(
    full_df[full_df.make.isin(makes_to_investigate)],
    x='price',
    y='predicted_price',
    color='age_at_posting',
    opacity=0.5,
    facet_col='make',
    facet_col_wrap=2,
)
fig.update_layout(
    xaxis_title="Actual Prices",
    yaxis_title="Predicted Prices",
    title="Scatter plot: Actual Prices vs. Predicted Prices",
    width=1000,
    height=500,
)
# plot a diagonal black line to show ideal diagonal perfect predictions
for i in range(2):
    fig.add_shape(
        type="line",
        x0=0,
        y0=0,
        x1=full_df[full_df.make.isin(makes_to_investigate)].price.max(),
        y1=full_df[full_df.make.isin(makes_to_investigate)].price.max(),
        line=dict(
            color="Black",
            width=3,
            dash="dashdot",
        ),
        row=1,
        col=i+1,
    )

fig.show()

In [ ]:
#create subplot with two rows
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# add MAPE bar chart to subplot
fig_mape = go.Bar(
    x=make_metrics['make'], 
    y=make_metrics['MAPE'], 
    name='MAPE',
    text = make_metrics['MAPE'].apply(lambda x: f"{x:.0f}%"),
)
fig.add_trace(fig_mape, row=1, col=1)

# add RMSE bar chart to subplot
fig_rmse = go.Bar(
    x=make_metrics['make'], 
    y=make_metrics['RMSE'], 
    name='RMSE',
    text = make_metrics['RMSE'].apply(lambda x: f"${x/1000:.1f}k")
)
fig.add_trace(fig_rmse, row=2, col=1)

# add count of the number of ads for each make to subplot
fig_count = go.Bar(
    x=full_df['make'].value_counts().index, 
    y=full_df['make'].value_counts(), 
    name='Count',
    # put text in k of ads
    text=full_df['make'].value_counts().apply(lambda x: f"{x/1000:.1f}k"),
)
fig.add_trace(fig_count, row=3, col=1)

# add the number of unique models for each make to subplot
fig_model_count = go.Bar(
    x=make_metrics['make'],
    y=make_metrics['make'].apply(lambda x: len(full_df[full_df['make'] == x]['model'].unique())),
    name='Unique Models',
    text=make_metrics['make'].apply(lambda x: len(full_df[full_df['make'] == x]['model'].unique())),
)
fig.add_trace(fig_model_count, row=4, col=1)

# update layout
fig.update_layout(title='MAPE and RMSE by Make',  
                  font=dict(size=14), 
                  height=1000)

# update y-axis titles
fig.update_yaxes(title_text='MAPE', row=1, col=1)
fig.update_yaxes(title_text='RMSE', row=2, col=1)
fig.update_yaxes(title_text= "No. of Ads", row=3, col = 1)
fig.update_yaxes(title_text= "No. of Unique Models", row=4, col = 1)
fig.update_xaxes(title_text="Make", row=1, col=1)

# reorder the shared x axes to sort by descending MAPE
fig.update_xaxes(categoryorder='array', categoryarray=make_metrics.sort_values(by='MAPE', ascending=False)['make'], row=1, col=1)
# fig.update_xaxes(categoryorder='array', categoryarray=make_metrics.sort_values(by='MAPE', ascending=False)['make'], row=2, col=1)
# fig.update_xaxes(categoryorder='array', categoryarray=make_metrics.sort_values(by='MAPE', ascending=False)['make'], row=3, col=1)

fig.show()

In [ ]:
# is there any correlation between MAPE or RMSE and number of ads by make?
make_num_ads_count = full_df['make'].value_counts().reset_index().rename(columns={'index': 'make', 'make': 'num_ads'}) 
make_metrics_with_num_ads = make_metrics.merge(make_num_ads_count, on='make')
make_metrics_with_num_ads[["MAPE", "RMSE", "num_ads"]].corr()

In [ ]:
# is there any correlation between MAPE or RMSE and number of ads by model?
model_num_ads_count = full_df['model'].value_counts().reset_index().rename(columns={'index': 'model', 'model': 'num_ads'}) 
model_metrics_with_num_ads = make_model_metrics.merge(model_num_ads_count, on='model')
model_metrics_with_num_ads[["MAPE", "RMSE", "num_ads"]].corr()

In [ ]:
# create bins for age_at_posting column
bins = pd.IntervalIndex.from_tuples([(-2, -1), (-1, 1), (1, 3), (3, 5), (5, 8), (8, 10), (10, 15), (15, 20)])

# create age_bins column
full_df['age_bins'] = pd.cut(full_df['age_at_posting'], bins)

# print value counts for age_bins column
print(full_df['age_bins'].value_counts())

In [ ]:
# how many of the ads are condition of new?
ads.df.condition.value_counts()

In [ ]:
# plot the distribution of mileage for cars condition = New
fig = px.histogram(
    ads.df[(ads.df.condition == 'New') & (ads.df.mileage < 100_000) & (ads.df.age_at_posting < 8)],
    x="mileage",
    title="Distribution of Mileage for Cars Condition = New",
    color="age_at_posting",
    labels={"mileage_per_year": "Mileage Per Year"},
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    # sort categroy by age_at_posting
    category_orders={"age_at_posting": np.sort(ads.df.age_at_posting.unique())},
    height=500,
    # set bin width to 1000
    nbins=200,
)
# set ylimit to 0  to 100
fig.update_yaxes(range=[0, 100])
fig.show()

## Learnings & Next Steps  

The key learnings from this analysis is:
- the training data preprocessing before had the following issues/bugs:
  - the minimum number of ads was applied before other criteria removed ads, meaning some makes/models had less than the minimum number of ads
  - there are many ads posted with condition New but their mileage is significantly different from zero and the cars age at posting is greater than zero
  - there are car condition categories such as Damaged, Lease takeover and not roadworthy that are not useful for the model
- some vehicle models have local minima seen as horizontal bands in the scatter plots where they're the same vehicle details but quite wide price ranges indicating we need more info to differentiate these vehicles
- there doesn't seem to be a significant correlation between number of ads and RMSE/MAPE, indicating we're not seeing and issue of not having enough data but instead feature engineering/details

Next Steps:
1. Update the training preprocessing script to fix the issues/bugs
   1. Add some logic to exclude new vehicles (negative age at a posting and/or condition new) and vehicles with condition categories that are not useful for the model
2. Add additional features like optional add ons to model training to try and improve model performance
2. Fix vehicle model/make issues/mapping corrections
3. Add model metrics by make/model reporting to model training loop
   1. Function to generate metrics by make/model
   2. Log metrics to MLflow as csv or similar during training

